In [29]:
# -*- coding: utf-8 -*-
"""
Created on Thu Feb 28 19:55:26 2019

@author: ericl
"""

import os
import librosa
import numpy as np
import scipy
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import pandas as pd
from tqdm import tqdm

clean_test_folder = r'F:\ECE271B_Project\Noise_Addition\results\test\babble\0db'
noisy_test_folder = r'F:\ECE271B_Project\Noise_Addition\results\test'
serialized_test_folder=r'F:\testset_results_data'
noisy_types=[r'\babble',r'\destroyerengine',r'\factory1',r'\hfchannel',r'\pink',r'\white']
SNRs=[r'\5db',r'\10db',r'\15db',r'\20db',r'\0db',r'\-5db']



window_size = 2 ** 14  # about 1 second of samples
#sample_rate = 16000


def saveConvert_info(file):
    """
    input a wav file, return np array after stft
    """
    sample_rate, samples = wavfile.read(file)
    x=scipy.signal.stft(samples,sample_rate)
    D_a = np.abs(x[2])
    D_db = librosa.core.amplitude_to_db(D_a, ref=np.max)
    phase=np.angle(x[2])
    mean=np.mean(D_db,1)
    std=np.std(D_db,1)
    max_value=np.max(D_a)
    return [phase,mean,std,max_value]

def saveConvert_data(file):
    """
    input a wav file, return np array after stft
    """
    sample_rate, samples = wavfile.read(file)
    x=scipy.signal.stft(samples,sample_rate)
    D_a = np.abs(x[2])

    D_db = librosa.core.amplitude_to_db(D_a, ref=np.max)
    return D_db
    
 
def normalize(data):
    return (data-np.mean(data,axis=1).reshape(129,1))/np.std(data,axis=1).reshape(129,1)

test_dict={}
def process_and_serialize(data_type):
    """
    Serialize, down-sample the sliced signals and save on separate folder.
    """
    for snr in SNRs:
        for noise in noisy_types:
            #max_idxs=[]
            
            if data_type == 'train':
                clean_folder = clean_train_folder
                noisy_folder = noisy_train_folder+noise+snr
                serialized_folder = serialized_train_folder+noise+snr
            else:
                clean_folder = clean_test_folder
                noisy_folder = noisy_test_folder+noise+snr
                serialized_folder = serialized_test_folder+noise+snr
            if not os.path.exists(serialized_folder):
                os.makedirs(serialized_folder)
            
            #clean_folder = clean_test_folder
            #noisy_folder = noisy_test_folder+noise+snr
            
            
            for root, dirs, files in os.walk(clean_folder):
                if len(files) == 0:
                    continue
                #print('current folder',dirs)
                for filename in tqdm(files, desc='Converting {} audios'.format(data_type)):
                    noisy_file = os.path.join(noisy_folder, filename)
                    if '.wav' in filename:
                    #[phase,mean,std,max_value]


                        data=saveConvert_data(noisy_file)
                        data=normalize(data)
                        (a,b,c,d)=saveConvert_info(noisy_file)
                        test_dict['phase']=a
                        test_dict['mean']=b
                        test_dict['std']=c
                        test_dict['max_value']=d
                        test_dict['data']=data
                        np.save(os.path.join(serialized_folder, '{}'.format(filename)),test_dict)
                        #np.save(os.path.join(serialized_folder, '{}'.format(filename)), arr=test_dict)
                        #print(noisy_file)

                        
            
            """
            for root, dirs, files in os.walk(clean_folder):
                for filename in tqdm(files, desc='Converting {} audios'.format(data_type)):
                    if '.wav' in filename:
                        noisy_file = os.path.join(noisy_folder, filename)
                        converted_noisy=saveConvert_data(noisy_file)
                        test=normalize(converted_noisy)
                        np.save(os.path.join(serialized_folder, '{}'.format(filename)), arr=test)
            

                    #print(np.sum(np.isnan(converted_noisy)))
                    #max_idxs.append((filename,converted_noisy.shape[1]))
                    
            print(serialized_folder)
            """
    

In [30]:
process_and_serialize("s")

Converting s audios: 100%|██████████████████████████████████████████████████████████| 201/201 [00:01<00:00, 111.84it/s]


In [17]:
import numpy as np

In [ ]:
np.angle